In [1]:
import tensorflow as tf
Dense = tf.keras.layers.Dense
import pandas as pd
import numpy as np
import h5py
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
tf.test.is_gpu_available()

True

In [3]:
data = pd.read_csv("dummy2018.csv", usecols=[i for i in range(1, 80)], dtype=int)

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.regularizers import l1, l2

In [6]:
model = Sequential()
model.add(LSTM(256, activation='tanh', return_sequences=True, input_shape=(256, 79)))
model.add(LSTM(256, activation='tanh', return_sequences=True, input_shape=(256, 79)))
model.add(LSTM(256, activation='tanh', return_sequences=True, input_shape=(256, 79)))
model.add(LSTM(256, activation='tanh', kernel_initializer='glorot_uniform', input_shape=(256, 79)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(79, activation='sigmoid'))

opt = RMSprop(lr=0.001, clipnorm=1.0)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256, 256)          344064    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256, 256)          525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1

In [7]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [8]:
data_gen = TimeseriesGenerator(data.values, data.values, length=256, stride=32, batch_size=32)

In [9]:
x, y = data_gen[0]

In [10]:
history = model.fit_generator(data_gen, epochs=1000, verbose=1)

W0705 13:07:52.215011 139855041312576 deprecation.py:323] From /home/brian/miniconda3/envs/tf2.0/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1000
84/84 [==============================] - 10s 119ms/step - loss: 0.5090 - accuracy: 0.9387
Epoch 2/1000
84/84 [==============================] - 6s 69ms/step - loss: 0.4291 - accuracy: 0.9546
Epoch 3/1000
84/84 [==============================] - 5s 57ms/step - loss: 0.4013 - accuracy: 0.9544
Epoch 4/1000
84/84 [==============================] - 6s 76ms/step - loss: 0.3823 - accuracy: 0.9563
Epoch 5/1000
84/84 [==============================] - 5s 56ms/step - loss: 0.4060 - accuracy: 0.9570
Epoch 6/1000
84/84 [==============================] - 5s 54ms/step - loss: 0.4424 - accuracy: 0.9574
Epoch 7/1000
84/84 [==============================] - 5s 55ms/step - loss: 0.3983 - accuracy: 0.9555
Epoch 8/1000
84/84 [==============================] - 5s 55ms/step - loss: 0.4188 - accuracy: 0.9567
Epoch 9/1000
84/84 [==============================] - 5s 55ms/step - loss: 0.4034 - accuracy: 0.9577
Epoch 10/1000
80/84 [===========================>..] - ETA: 0s - loss: 0.3665 - accuracy:

KeyboardInterrupt: 

In [ ]:
model.save("test-lstm-1.h5")